In [1]:
import pandas as pd
import numpy as np
from datetime import date

Tratando as bases

In [44]:
dict_moedas = ['ZAR', 'AFS', 'AUD', 'AUS', 'ARB', 'ARS', 'CAD', 'CAN', 'CLP', 
'CHL', 'CNY', 'CNH', 'DOL', 'EUR', 'EUP', 'GBP', 'GBR', 'JPY', 'JAP', 'MXN', 'MEX',
'NOK', 'NZD', 'NZL', 'RUB', 'SEK', 'CHF', 'SWI', 'TRY', 'TUQ']

In [10]:
# df=pd.read_csv('ft_moedas_1.0.csv',delimiter=',')
df=pd.read_csv('https://raw.githubusercontent.com/iqfc/B3_Scrapper/main/ft_moedas_1.0.csv',delimiter=',',low_memory=False)
df=df.assign(Contr_venc=df.Contract+'_'+df['Contract Months'])

df['Settlement Price']=df['Settlement Price'].str.replace(',','')
df['Settlement Price']=pd.to_numeric(df['Settlement Price'])

df['Financial Volume (R$)']=df['Financial Volume (R$)'].str.replace(',','')
df['Financial Volume (R$)']=pd.to_numeric(df['Financial Volume (R$)'])

Volume de transações

In [53]:
ft_vol=pd.DataFrame([df.Date,df['Contract Months'],df['Contract'],df['Financial Volume (R$)']]).T
ft_vol.Date=pd.to_datetime(ft_vol.Date).dt.date

Settlement price dos futuros

In [60]:
ft_price=pd.DataFrame([df.Date,df['Contract Months'],df['Contract'],df['Settlement Price']]).T
ft_price.Date=pd.to_datetime(ft_vol.Date)

Importando Spot Prices

In [52]:
# spot=pd.read_csv('CSV_moedas_1.csv',delimiter=',').set_index('Unnamed: 0')
spot=pd.read_csv('https://raw.githubusercontent.com/iqfc/B3_Scrapper/main/CSV_moedas_1.csv',delimiter=',')
spot['DOL']=spot.BRL
spot['Unnamed: 0']=pd.to_datetime(spot['Unnamed: 0']).dt.date
spot=spot.set_index('Unnamed: 0')
spot.index.name='Date'


Geting only the contracts available in all bases

In [36]:
contracts_available=pd.DataFrame(spot.columns).merge(pd.DataFrame(df.Contract.unique()),how='inner')
contracts_available=list(contracts_available[0])

spot=spot.loc[:,contracts_available]
ft_price=ft_price[ft_price.Contract.isin(contracts_available)]
ft_vol=ft_vol[ft_vol.Contract.isin(contracts_available)]

Backtest Structure Here

In [58]:
#Detalhe importante, para comparar datas usando o pandas agora precisamos colocar tudo no formado do proprio pandas, diferente do
#que faziamos antes no datetime.date, ou apenas o date, assim o startdate e end_date devem estar 


start_date=pd.Timestamp(2006,1,1)
end_date=date(2022,12,31)

rebal=1

#não precisa de lookback pois visamos pegar a data mais atual


data_analise=start_date-pd.DateOffset(days=5)

#Presiso dar o slice nas 3 bases ftvol, ft price e spot




In [43]:
price=spot.loc[data_analise:start_date]

In [61]:
futures=ft_price[(ft_price.Date > data_analise) & (ft_price.Date < start_date)]